In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os


In [ ]:
# Setup paths for local execution
import os
import sys

# Project directories
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
DATA_DIR = os.path.join(BASE_DIR, 'data')
OUTPUT_DIR = os.path.join(BASE_DIR, 'outputs')
SRC_DIR = os.path.join(BASE_DIR, 'src')

# Add src to path
sys.path.insert(0, SRC_DIR)


# Mounting drive and reading file

Mounted at /content/drive/


In [9]:
data1 = pd.read_csv('../../data/stock_week_cluster.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
data1.drop(['Unnamed: 0'],axis=1,inplace=True)

In [11]:
data1['ds'].max()

'2011-10-31'

In [12]:
data1['ds'] = pd.to_datetime(data1['ds'])

# Creating feature like week of month, year, week-number and Month

In [13]:
def weekinmonth(x):
  x = pd.to_datetime(x)
  fdom = x.day // 7
  return fdom
data1['wom'] = data1['ds'].apply(weekinmonth)
data1['year'] = data1['ds'].dt.year
data1['week_no'] = data1['ds'].dt.week
# data1['month'] = data1['ds'].dt.month

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  import sys


# removing data for year 2009

In [14]:
data1 = data1.loc[data1['year'] >= 2010,]

In [15]:
data1.shape

(305914, 13)

In [16]:
dtgc = data1.copy()

In [17]:
dtgc.head()

,StockCode,Quantity,Price,holiday,ds,year,timeelapsed,cumsum,cumdist,averagedistance,cluster,wom,week_no
0,20677,96.000000,1.583599,-1.0,2010-05-31,2010,1.0,96.000000,0.0,0.013333,9,4,22
1,20677,10.500000,1.583599,-1.0,2010-06-07,2010,1.0,106.500000,0.0,0.013333,9,1,23
2,20677,32.000000,1.583599,-1.0,2010-06-14,2010,1.0,138.500000,0.0,0.013333,9,2,24
3,20677,30.833333,1.583599,-1.0,2010-06-21,2010,1.0,169.333333,0.0,0.013333,9,3,25
4,20677,123.666667,1.583599,-1.0,2010-06-28,2010,1.0,293.000000,0.0,0.013333,9,4,26


In [18]:
dtgc.shape

(305914, 13)

In [19]:
dtgc.head()

,StockCode,Quantity,Price,holiday,ds,year,timeelapsed,cumsum,cumdist,averagedistance,cluster,wom,week_no
0,20677,96.000000,1.583599,-1.0,2010-05-31,2010,1.0,96.000000,0.0,0.013333,9,4,22
1,20677,10.500000,1.583599,-1.0,2010-06-07,2010,1.0,106.500000,0.0,0.013333,9,1,23
2,20677,32.000000,1.583599,-1.0,2010-06-14,2010,1.0,138.500000,0.0,0.013333,9,2,24
3,20677,30.833333,1.583599,-1.0,2010-06-21,2010,1.0,169.333333,0.0,0.013333,9,3,25
4,20677,123.666667,1.583599,-1.0,2010-06-28,2010,1.0,293.000000,0.0,0.013333,9,4,26


# Standardization by Stock Code. it is (x-mean)/sd and lag feature creation

In [20]:
mu = data1.groupby(['StockCode'])['Quantity'].mean().reset_index()
mu = mu.rename(columns={'Quantity':'mean'})
std = dtgc.groupby(['StockCode'])['Quantity'].std().reset_index()
std = std.rename(columns={'Quantity':'std'})
data1 = dtgc.merge(mu,on=['StockCode'],how='left')
data1 = data1.merge(std,on=['StockCode'],how='left')
data1['tQuantity'] = (data1['Quantity'] - data1['mean'])/(data1['std'])
data = data1.copy()
data['lag1'] = data.groupby('StockCode')['tQuantity'].shift(1)
data['lag4'] = data.groupby('StockCode')['tQuantity'].shift(4)
data['lag52'] = data.groupby('StockCode')['tQuantity'].shift(52)


In [21]:
data.isnull().sum()

StockCode               0
Quantity                0
Price                   0
holiday               612
ds                      0
year                    0
timeelapsed             0
cumsum                  0
cumdist                 0
averagedistance         0
cluster                 0
wom                     0
week_no                 0
mean                    0
std                     0
tQuantity              96
lag1                 3553
lag4                13924
lag52              175238
dtype: int64

In [22]:
# data.drop(['cumsum'],axis=1,inplace=True)

# Replacing missing value with -1. 

In [23]:
data.fillna(-1,inplace = True)

# Train val test split

In [24]:
max_date = pd.to_datetime(data['ds']).max()
mx1_dt = (max_date - pd.to_timedelta(7, unit='w'))

In [25]:
data['ds'] = pd.to_datetime(data['ds'])
train = data.loc[data['ds'] <= mx1_dt,]

In [26]:
val = data.loc[data['ds'] > mx1_dt,]
val_f = val[[x for x in val.columns if x not in ['Quantity','mean','qtr','cumsum','timeelapsed']]]


In [27]:
train_f = train[[x for x in train.columns if x not in ['Quantity','mean','qtr','cumsum','timeelapsed']]]


In [28]:
mx_dt = (train['ds'].max() - pd.to_timedelta(6, unit='w'))

In [29]:
train1 = train.loc[train['ds'] <= mx_dt,]
val1 = train.loc[train['ds'] >  mx_dt,]
test = data.loc[data['ds'] > val1['ds'].max(),]

In [30]:
test['ds'].max()

Timestamp('2011-10-31 00:00:00')

In [31]:
train_x = train1[[x for x in train1.columns if x not in ['Quantity','mean','qtr','cumsum','timeelapsed']]]


# Required Package installation & Stock-Code ordinal encoding

In [32]:
! pip install category_encoders

In [33]:
import category_encoders as ce
id_encoder =  ce.OrdinalEncoder()
id_encoder.fit(train_x['StockCode'])
train_x['id_cat'] = id_encoder.transform(train_x['StockCode'])
train_x1 = train_x.drop(['StockCode'],axis=1)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [34]:
val_x = val1[[x for x in val1.columns if x not in ['Quantity','mean','qtr','cumsum','timeelapsed']]]


In [35]:
val_x['id_cat'] = id_encoder.transform(val_x['StockCode'])
val_x1 = val_x.drop(['StockCode'],axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
test_x = test[[x for x in test.columns if x not in ['Quantity','mean','qtr','cumsum','timeelapsed']]]
test_x['id_cat'] = id_encoder.transform(test_x['StockCode'])
test_x1 = test_x.drop(['StockCode'],axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [37]:
train_f['id_cat'] = id_encoder.transform(train_f['StockCode'])
train_f = train_f.drop(['StockCode'],axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [38]:
val_f['id_cat'] = id_encoder.transform(val_f['StockCode'])
val_f.drop(['StockCode'],axis=1,inplace=True)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [39]:
val_f.head()

,Price,holiday,ds,year,cumdist,averagedistance,cluster,wom,week_no,std,tQuantity,lag1,lag4,lag52,id_cat
68,1.583599,-1.0,2011-09-19,2011,0.0,0.013333,9,2,38,40.057125,1.765346,-0.984893,-0.223481,-0.835107,1.0
69,1.583599,-1.0,2011-09-26,2011,0.0,0.013333,9,3,39,40.057125,-0.323338,1.765346,-0.435678,-0.627071,1.0
70,1.583599,-1.0,2011-10-03,2011,0.0,0.013333,9,0,40,40.057125,2.759759,-0.323338,-0.572982,-0.785179,1.0
71,1.583599,-1.0,2011-10-10,2011,0.0,0.013333,9,1,41,40.057125,-0.910000,2.759759,-0.984893,-0.772696,1.0
72,1.583599,-1.0,2011-10-17,2011,0.0,0.013333,9,2,42,40.057125,-0.664518,-0.910000,1.765346,-0.785179,1.0


#  Deep Future Architecture. Just used some portion of it as full architecture is not designed to meet this requirement of forecasting intermittant series

In [40]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '/content/drive/My Drive/ML_AI/DeepFuture/')

# installing tensorflow lattice

In [41]:
! pip install tensorflow_lattice

# Required package installation

In [42]:
import FourierComponent as FC
import RegressorComponent as RC
import seasonal_component as SC
import pandas as pd
import numpy as np
import tensorflow as tf
from additive_multiplicative import Seaonal_Component as scm
from additive_multiplicative import Regressor_Component as rcm
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import plot_model

In [43]:
def swish(x):
    return x*K.sigmoid(x)
def listh(x):
    return x*K.tanh(x)
def mish(x):
    return x*K.tanh(K.softplus(x))

In [44]:
custom_object= {'swish':swish, 'mish':mish,'listh':listh,'sigmoid':'sigmoid','relu':'relu'}

In [59]:
categorical_feature = ['week_no','wom','cluster']

In [46]:
train_x1.head()

,Price,holiday,ds,year,cumdist,averagedistance,cluster,wom,week_no,std,tQuantity,lag1,lag4,lag52,id_cat
0,1.583599,-1.0,2010-05-31,2010,0.0,0.013333,9,4,22,40.057125,1.261898,-1.000000,-1.000000,-1.0,1
1,1.583599,-1.0,2010-06-07,2010,0.0,0.013333,9,1,23,40.057125,-0.872554,1.261898,-1.000000,-1.0,1
2,1.583599,-1.0,2010-06-14,2010,0.0,0.013333,9,2,24,40.057125,-0.335820,-0.872554,-1.000000,-1.0,1
3,1.583599,-1.0,2010-06-21,2010,0.0,0.013333,9,3,25,40.057125,-0.364945,-0.335820,-1.000000,-1.0,1
4,1.583599,-1.0,2010-06-28,2010,0.0,0.013333,9,4,26,40.057125,1.952578,-0.364945,1.261898,-1.0,1


In [47]:
train_x2 = train_x1[[x for x in train_x1.columns if x in ['id_cat','ds','tQuantity']]]

In [48]:
exog = train_x1[[x for x in train_x1.columns if x not in ['id_cat','ds','tQuantity']]]

In [49]:
val_x2 = val_x1[[x for x in val_x1.columns if x in ['id_cat','ds','tQuantity']]]
val_exog = val_x1[[x for x in val_x1.columns if x not in ['id_cat','ds','tQuantity']]]

In [61]:
context_variable = [x for x in exog.columns if x not in categorical_feature]

constraint = [None for x in context_variable]

In [62]:
exog.columns

Index(['Price', 'holiday', 'year', 'cumdist', 'averagedistance', 'cluster',
       'wom', 'week_no', 'std', 'lag1', 'lag4', 'lag52'],
      dtype='object')

# Function to create input for model

In [63]:
def model_input(model,id_var='id_cat',exog=exog,val=False,val_ts=None):
  s_input = []
  r_input = []
  if not val:
    for input in model[0].sr_df.columns:
      if input !='ds':
        s_input.append(model[0].sr_df[input].values)
    
  else:
    val_sr_df = model[0].seasonal_feature(val_ts)
    for input in val_sr_df.columns:
      if input !='ds':
        s_input.append(val_sr_df[input].values)
  for input in model[1].combined_reg_model.input_names:
      r_input.append(exog[input].values)
    
  return [s_input,r_input]

# Function to be minimised while tuning hyper param. trying with just 5 trial just for initial guess.other wise it would take lot more time

In [64]:
# Func

In [65]:
def validation(dat,dat2,val_exog,val_ts,target,id_var,categorical_var,context_variable, constraint,loss_fn,seasonal_hl,seasonal_hunit,seasonality_l1,seasonal_dropout,
               sr_hidden_act,sr_output_act, rr_hidden_act,rr_output_act,rm_lat_unit,rm_lattice_size,rm_hidden_unit,
              rm_hidden_layer,rm_drop_out,rm_L1,batch_size=512,path='../../outputs/'):
  sm =  SC.seasonal(dat,target=target,id_var=id_var, horizon=8,k=None,daily=False,weekly=True, yearly=True,monthly= True,qtr=False,period=None,unit='w')
  sm.seasonal_feature()
  sm.seasonal_model(id_input=None,hidden =seasonal_hl,hidden_unit=seasonal_hunit, hidden_act=sr_hidden_act,output_act=sr_output_act,reg=seasonality_l1,
                    embed_size=50,drop_out=seasonal_dropout)
  
  rm = RC.Regressor(ts=dat,exog=dat2,target=target,id_var=id_var,categorical_var=categorical_var,context_variable=context_variable, constraint=constraint,embed_size=50,
                    lat_unit=rm_lat_unit,lattice_size=rm_lattice_size,hidden_unit=rm_hidden_unit,hidden_act=swish, output_act=swish,
                    hidden_layer=rm_hidden_layer,drop_out=rm_drop_out,L1=rm_L1,rnge=.8)
  train_y = dat[target].values
  val_y = val_ts[target].values
  rm.reg_model(id_input=sm.s_model.input[-1])
  rc = rcm('additive')
  rm2 = rc.Regressor_comp(sm.s_model.output,rm.combined_reg_model)
  
  f_model = Model([sm.s_model.input,rm.combined_reg_model.input],rm2)
  check_point_path = path
  lr=0.001
  f_model.compile(loss=tf.keras.losses.MeanAbsolutePercentageError(),optimizer=tf.keras.optimizers.Adam(learning_rate=lr))
  call_back_ = [ModelCheckpoint(check_point_path,monitor= 'val_loss',save_best_only=True, mode='min'),
              EarlyStopping(monitor='val_loss', mode='min', patience=4)
              ]
  train_input = model_input([sm,rm],id_var=id_var,exog=dat2)
  val_input = model_input([sm,rm],exog=val_exog,val=True,val_ts=val_ts)
  hist= f_model.fit(train_input,train_y,batch_size=512,epochs=500,validation_data=(val_input,val_y),callbacks=call_back_)
  print(hist.history['val_loss'][-1])
  return hist


  

# installing optuna

In [66]:
! pip install optuna

In [67]:
import optuna

# tuning hyper param

In [68]:
def objective(trial):


  holidays_l1 = trial.suggest_float('holidays_l1', 0.0005, .05)
  seasonality_l1 = trial.suggest_float('seasonality_l1',0.0005, .05)
  seasonal_dropout = trial.suggest_categorical('seasonal_dropout', [0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45, 0.5])
  seasonal_hunit = trial.suggest_categorical('seasonal_hunit', [4,8,16])
  seasonal_hl = trial.suggest_int('seasonal_hl', 0, 4)
  sr_hidden_act = trial.suggest_categorical('sr_hidden_act', ['swish','mish','listh','sigmoid','relu'])
  sr_output_act = trial.suggest_categorical('sr_output_act', ['swish','mish','listh'])
  rr_hidden_act = trial.suggest_categorical('rr_hidden_act', ['swish','mish','listh','sigmoid','relu'])
  rr_output_act = trial.suggest_categorical('rr_output_act', ['swish','mish','listh','relu'])
  rm_lunit = trial.suggest_categorical('rm_lunit', [4,8,16])
  rm_lsize = trial.suggest_int('rm_lsize', 2, 4)
  rm_hl = trial.suggest_int('rm_hl', 0, 4)
  rm_hunit = trial.suggest_categorical('rm_hunit', [4,8,16])
  rm_dropout = trial.suggest_categorical('rm_dropout', [0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45, 0.5])
  rm_L1 = trial.suggest_float('rm_L1', 0.0005,.05)
  hist = validation(train_x1,exog,val_exog,val_x2,['tQuantity'],'id_cat',categorical_feature,context_variable, constraint,tf.keras.losses.MeanAbsolutePercentageError(),seasonal_hl,
                    seasonal_hunit,seasonality_l1,seasonal_dropout, custom_object[sr_hidden_act], custom_object[sr_output_act],custom_object[rr_hidden_act]
                    ,custom_object[rr_output_act],rm_lunit,rm_lsize,rm_hunit,rm_hl,rm_dropout,rm_L1,batch_size=512,
                    path='../../outputs/')
  return hist.history['val_loss'][-1]


In [69]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,  n_trials=5, n_jobs=-1)

[I 2021-02-22 13:41:28,116] A new study created in memory with name: no-name-6d40ad5f-2e57-4cb9-bfef-1cc845b63e8a


[26.  2.]
[26.  2.]
   id_cat         ds
0       1 2011-08-01
1       2 2011-08-01
2       3 2011-08-01
3       4 2011-08-01
4       5 2011-08-01   id_cat         ds
0       1 2011-08-01
1       2 2011-08-01
2       3 2011-08-01
3       4 2011-08-01
4       5 2011-08-01

   id_cat         ds
0       1 2011-08-01
1       2 2011-08-01
2       3 2011-08-01
3       4 2011-08-01
4       5 2011-08-01
   id_cat         ds
0       1 2011-08-01
1       2 2011-08-01
2       3 2011-08-01
3       4 2011-08-01
4       5 2011-08-01
fts value        id_cat         ds
0           1 2011-08-08
1           1 2011-08-15
2           1 2011-08-22
3           1 2011-08-29
4           1 2011-09-05
...       ...        ...
27651    3457 2011-08-29
27652    3457 2011-09-05
27653    3457 2011-09-12
27654    3457 2011-09-19
27655    3457 2011-09-26

[27656 rows x 2 columns]
fts value        id_cat         ds
0           1 2011-08-08
1           1 2011-08-15
2           1 2011-08-22
3           1 2011-08-29
4    

/content/drive/My Drive/ML_AI/DeepFuture/RegressorComponent.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



KerasTensor(type_spec=TensorSpec(shape=(None, 27), dtype=tf.float32, name=None), name='reshape_1/Reshape:0', description="created by layer 'reshape_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 50), dtype=tf.float32, name=None), name='reshape/Reshape:0', description="created by layer 'reshape'")
KerasTensor(type_spec=TensorSpec(shape=(None, 3), dtype=tf.float32, name=None), name='reshape_2/Reshape:0', description="created by layer 'reshape_2'")
KerasTensor(type_spec=TensorSpec(shape=(None, 9), dtype=tf.float32, name=None), name='reshape_3/Reshape:0', description="created by layer 'reshape_3'")
166
Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 1

/content/drive/My Drive/ML_AI/DeepFuture/RegressorComponent.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



...

[26.  2.]
   id_cat         ds
0       1 2011-08-01
1       2 2011-08-01
2       3 2011-08-01
3       4 2011-08-01
468/510 [==========================>...] - ETA: 5s - loss: 2405.6358   id_cat         ds
0       1 2011-08-01
1       2 2011-08-01
2       3 2011-08-01
3       4 2011-08-01
4       5 2011-08-01
492/510 [===========================>..] - ETA: 2s - loss: 2514.4553fts value        id_cat         ds
0           1 2011-08-08
1           1 2011-08-15
2           1 2011-08-22
3           1 2011-08-29
4           1 2011-09-05
...       ...        ...
27651    3457 2011-08-29
27652    3457 2011-09-05
27653    3457 2011-09-12
27654    3457 2011-09-19
27655    3457 2011-09-26

495/510 [============================>.] - ETA: 1s - loss: 2529.92983457
KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.float32, name='id_cat'), name='id_cat', description="created by layer 'id_cat'")
KerasTensor(type_spec=TensorSpec(shape=(None, 50), dtype=tf.float32, name=None), name='reshape_5/Re

[I 2021-02-22 14:17:13,896] Trial 3 finished with value: 127.28328704833984 and parameters: {'holidays_l1': 0.029152002071007394, 'seasonality_l1': 0.018822968761065546, 'seasonal_dropout': 0.35, 'seasonal_hunit': 16, 'seasonal_hl': 0, 'sr_hidden_act': 'relu', 'sr_output_act': 'swish', 'rr_hidden_act': 'swish', 'rr_output_act': 'mish', 'rm_lunit': 16, 'rm_lsize': 4, 'rm_hl': 1, 'rm_hunit': 8, 'rm_dropout': 0.4, 'rm_L1': 0.0036222025224323905}. Best is trial 0 with value: 117.24148559570312.


127.28328704833984
510/510 [==============================] - 15s 29ms/step - loss: 11543.0137 - val_loss: 126.7054
INFO:tensorflow:Assets written to: //content/drive/My Drive/jubilant/assets
Epoch 11/500
510/510 [==============================] - 10s 20ms/step - loss: 13832.1010 - val_loss: 126.5307
INFO:tensorflow:Assets written to: //content/drive/My Drive/jubilant/assets
Epoch 12/500
510/510 [==============================] - 10s 19ms/step - loss: 3008.9888 - val_loss: 135.4021
Epoch 13/500
510/510 [==============================] - 10s 19ms/step - loss: 3087.9136 - val_loss: 126.3725
INFO:tensorflow:Assets written to: //content/drive/My Drive/jubilant/assets
Epoch 14/500
510/510 [==============================] - 10s 19ms/step - loss: 2769.1390 - val_loss: 125.6062
INFO:tensorflow:Assets written to: //content/drive/My Drive/jubilant/assets
Epoch 15/500
510/510 [==============================] - 10s 19ms/step - loss: 3564.9005 - val_loss: 121.7757
INFO:tensorflow:Assets written to:

[I 2021-02-22 14:19:53,881] Trial 4 finished with value: 126.84563446044922 and parameters: {'holidays_l1': 0.02844144349793173, 'seasonality_l1': 0.014146608619066695, 'seasonal_dropout': 0.25, 'seasonal_hunit': 16, 'seasonal_hl': 0, 'sr_hidden_act': 'listh', 'sr_output_act': 'mish', 'rr_hidden_act': 'sigmoid', 'rr_output_act': 'swish', 'rm_lunit': 4, 'rm_lsize': 3, 'rm_hl': 4, 'rm_hunit': 16, 'rm_dropout': 0.45, 'rm_L1': 0.023091933630671597}. Best is trial 0 with value: 117.24148559570312.


126.84563446044922


# getting inital guessed param

In [70]:

best_param = study.best_params

In [81]:
best_param = {'holidays_l1': 0.03258044288465095,
 'rm_L1': 0.032145693506182474,
 'rm_dropout': 0.1,
 'rm_hl': 1,
 'rm_hunit': 4,
 'rm_lsize': 4,
 'rm_lunit': 4,
 'rr_hidden_act': 'mish',
 'rr_output_act': 'listh',
 'seasonal_dropout': 0.1,
 'seasonal_hl': 1,
 'seasonal_hunit': 4,
 'seasonality_l1': 0.011003144247890326,
 'sr_hidden_act': 'mish',
 'sr_output_act': 'swish'}

In [82]:

seasonality_l1 = best_param['seasonality_l1']
seasonal_dropout = best_param['seasonal_dropout']
seasonal_hunit = best_param['seasonal_hunit']
seasonal_hl = best_param['seasonal_hl']
sr_hidden_act = best_param['sr_hidden_act']
sr_output_act = best_param['sr_output_act']
rr_hidden_act = best_param['rr_hidden_act']
rr_output_act = best_param['rr_output_act']
rm_lunit = best_param['rm_lunit']
rm_lsize = best_param['rm_lsize']
rm_hl = best_param['rm_hl']
rm_hunit = best_param['rm_hunit']
rm_dropout = best_param['rm_dropout']
rm_L1 = best_param['rm_L1']
    

In [83]:
# custom_object= {'swish':swish, 'mish':mish,'listh':listh,'q_loss':qloss}
# model = tf.keras.models.load_model('../../outputs/',custom_objects=custom_object)

# Retraining with initial guessed param estimate

In [84]:
# exog

In [85]:
custom_object= {'swish':swish, 'mish':mish,'listh':listh,'sigmoid':'sigmoid','relu':'relu'}
sm =  SC.seasonal(train_x1,target=['tQuantity'],id_var='id_cat', horizon=8,k=None,daily=False,weekly=True, yearly=True,monthly= True,qtr=False,period=None,unit='w')
sm.seasonal_feature()
sm.seasonal_model(id_input=None,hidden =seasonal_hl,hidden_unit=seasonal_hunit, hidden_act=custom_object[sr_hidden_act],output_act=custom_object[sr_output_act],reg=seasonality_l1,
                  embed_size=50,drop_out=seasonal_dropout)

rm = RC.Regressor(ts=train_x1,exog=exog,target=['tQuantity'],id_var='id_cat',categorical_var=categorical_feature,context_variable=context_variable, constraint=constraint,embed_size=50,
                  lat_unit=rm_lunit,lattice_size=rm_lsize,hidden_unit=rm_hunit,hidden_act=custom_object[rr_hidden_act], output_act=custom_object[rr_output_act],
                  hidden_layer=rm_hl,drop_out=rm_dropout,L1=rm_L1,rnge=.8)
train_y = train_x1['tQuantity'].values
val_y = val_x1['tQuantity'].values
rm.reg_model(id_input=sm.s_model.input[-1])
rc = rcm('additive')
rm2 = rc.Regressor_comp(sm.s_model.output,rm.combined_reg_model)
f_model = Model([sm.s_model.input,rm.combined_reg_model.input],rm2)
check_point_path = path='../../outputs/'
lr=0.001
f_model.compile(loss=tf.keras.losses.MeanAbsolutePercentageError(),optimizer=tf.keras.optimizers.Adam(learning_rate=lr))
call_back_ = [ModelCheckpoint(check_point_path,monitor= 'val_loss',save_best_only=True, mode='min'),
            EarlyStopping(monitor='val_loss', mode='min', patience=10)
            ]
train_input = model_input([sm,rm],id_var='id_cat',exog=exog)
val_input = model_input([sm,rm],id_var='id_cat',exog=val_exog,val=True,val_ts=val_x2)


[26.  2.]
   id_cat         ds
0       1 2011-08-01
1       2 2011-08-01
2       3 2011-08-01
3       4 2011-08-01
4       5 2011-08-01
   id_cat         ds
0       1 2011-08-01
1       2 2011-08-01
2       3 2011-08-01
3       4 2011-08-01
4       5 2011-08-01
fts value        id_cat         ds
0           1 2011-08-08
1           1 2011-08-15
2           1 2011-08-22
3           1 2011-08-29
4           1 2011-09-05
...       ...        ...
27651    3457 2011-08-29
27652    3457 2011-09-05
27653    3457 2011-09-12
27654    3457 2011-09-19
27655    3457 2011-09-26

[27656 rows x 2 columns]
3457
KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.float32, name='id_cat'), name='id_cat', description="created by layer 'id_cat'")
KerasTensor(type_spec=TensorSpec(shape=(None, 50), dtype=tf.float32, name=None), name='reshape_10/Reshape:0', description="created by layer 'reshape_10'")
Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 

In [86]:
hist = f_model.fit(train_input,train_y,validation_data=(val_input,val_y),epochs=500,batch_size=2048,callbacks=call_back_)

Epoch 1/500
128/128 [==============================] - 13s 67ms/step - loss: 76133.9425 - val_loss: 216.2201
INFO:tensorflow:Assets written to: //content/drive/My Drive/jubilant/assets
Epoch 2/500
128/128 [==============================] - 7s 56ms/step - loss: 62092.4779 - val_loss: 242.9855
Epoch 3/500
128/128 [==============================] - 6s 50ms/step - loss: 21957.4824 - val_loss: 207.9379
INFO:tensorflow:Assets written to: //content/drive/My Drive/jubilant/assets
Epoch 4/500
128/128 [==============================] - 6s 49ms/step - loss: 27532.8492 - val_loss: 256.0867
Epoch 5/500
128/128 [==============================] - 6s 49ms/step - loss: 6853.3160 - val_loss: 351.9356
Epoch 6/500
128/128 [==============================] - 6s 50ms/step - loss: 13240.9622 - val_loss: 439.9780
Epoch 7/500
128/128 [==============================] - 6s 50ms/step - loss: 41683.7156 - val_loss: 217.7407
Epoch 8/500
128/128 [==============================] - 6s 49ms/step - loss: 5733.9377 - val_

# function to create input for recursive forecast

In [87]:
def rec_model_input(id_var='id_cat',exog=exog,val=True,val_ts=None,date=None):
  s_input = []
  r_input = []
  
  if not val:
    if date is not None:
      pr_sr = sm.pr_sr_df.loc[sm.pr_sr_df['ds'] == date,]
      
      for input in pr_sr.columns:
        if input !='ds':
          s_input.append(pr_sr[input].values)
    else:
      for input in pr_sr.columns:
        if input !='ds':
          s_input.append(sm.pr_sr_df[input].values)

    
  else:
    if val_ts is None:
      raise ValueError(('val_ts could not be None if val is True'))
    val_sr_df = sm.seasonal_feature(val_ts)
    
    for input in val_sr_df.columns:
      if input !='ds':
        s_input.append(val_sr_df[input].values)
  for input in rm.combined_reg_model.input_names:
      r_input.append(exog[input].values)
    
  return [s_input,r_input]

# Input preparation of recursive forecast

In [88]:
val1['forecast'] = f_model.predict(val_input)

In [89]:
train_x.tail(1)

,StockCode,Price,holiday,ds,year,cumdist,averagedistance,cluster,wom,week_no,std,tQuantity,lag1,lag4,lag52,id_cat
305900,gift_0001_30,25.420417,-1.0,2011-08-01,2011,9.0,3.34375,14,0,31,1.979272,-0.29209,-0.29209,-0.29209,-0.29209,3457


In [90]:
val1.tail(1)

,StockCode,Quantity,Price,holiday,ds,year,timeelapsed,cumsum,cumdist,averagedistance,cluster,wom,week_no,mean,std,tQuantity,lag1,lag4,lag52,forecast
305906,gift_0001_30,0.0,25.420417,-1.0,2011-09-12,2011,1.0,54.5,2.0,3.34375,14,1,37,0.578125,1.979272,-0.29209,-0.29209,-0.29209,-0.29209,-0.267007


In [91]:
train1['forecast'] = f_model.predict(train_input)

In [92]:
train_val = pd.concat([train1,val1],axis=0)

In [93]:
vale = train_val.copy()
vale.head()

,StockCode,Quantity,Price,holiday,ds,year,timeelapsed,cumsum,cumdist,averagedistance,cluster,wom,week_no,mean,std,tQuantity,lag1,lag4,lag52,forecast
0,20677,96.000000,1.583599,-1.0,2010-05-31,2010,1.0,96.000000,0.0,0.013333,9,4,22,45.452,40.057125,1.261898,-1.000000,-1.000000,-1.0,-0.077117
1,20677,10.500000,1.583599,-1.0,2010-06-07,2010,1.0,106.500000,0.0,0.013333,9,1,23,45.452,40.057125,-0.872554,1.261898,-1.000000,-1.0,-0.193786
2,20677,32.000000,1.583599,-1.0,2010-06-14,2010,1.0,138.500000,0.0,0.013333,9,2,24,45.452,40.057125,-0.335820,-0.872554,-1.000000,-1.0,-0.108649
3,20677,30.833333,1.583599,-1.0,2010-06-21,2010,1.0,169.333333,0.0,0.013333,9,3,25,45.452,40.057125,-0.364945,-0.335820,-1.000000,-1.0,-0.124395
4,20677,123.666667,1.583599,-1.0,2010-06-28,2010,1.0,293.000000,0.0,0.013333,9,4,26,45.452,40.057125,1.952578,-0.364945,1.261898,-1.0,-0.137025


In [94]:
vale['forecast'].max()

1.2397427558898926

In [95]:
vale = vale.reset_index()

In [96]:
for lag in range(2,5):
  vale['lag'+str(lag)] = vale.groupby(['StockCode'])['tQuantity'].shift(lag)

In [97]:
# train_val.index.day_name()

In [98]:
# train_val['ds']= train_val.index
# train_val = train_val.reset_index(drop=True)

In [99]:
train_val.head()

,StockCode,Quantity,Price,holiday,ds,year,timeelapsed,cumsum,cumdist,averagedistance,cluster,wom,week_no,mean,std,tQuantity,lag1,lag4,lag52,forecast
0,20677,96.000000,1.583599,-1.0,2010-05-31,2010,1.0,96.000000,0.0,0.013333,9,4,22,45.452,40.057125,1.261898,-1.000000,-1.000000,-1.0,-0.077117
1,20677,10.500000,1.583599,-1.0,2010-06-07,2010,1.0,106.500000,0.0,0.013333,9,1,23,45.452,40.057125,-0.872554,1.261898,-1.000000,-1.0,-0.193786
2,20677,32.000000,1.583599,-1.0,2010-06-14,2010,1.0,138.500000,0.0,0.013333,9,2,24,45.452,40.057125,-0.335820,-0.872554,-1.000000,-1.0,-0.108649
3,20677,30.833333,1.583599,-1.0,2010-06-21,2010,1.0,169.333333,0.0,0.013333,9,3,25,45.452,40.057125,-0.364945,-0.335820,-1.000000,-1.0,-0.124395
4,20677,123.666667,1.583599,-1.0,2010-06-28,2010,1.0,293.000000,0.0,0.013333,9,4,26,45.452,40.057125,1.952578,-0.364945,1.261898,-1.0,-0.137025


In [100]:
# test_x1['id_cat'] = id_encoder.transform(test['StockCode'])

In [101]:
test['id_cat'] = id_encoder.transform(test['StockCode'])

In [102]:
sm.pr_sr_df.head()

,S_0_52.17857142857143,C_1_52.17857142857143,S_2_52.17857142857143,C_3_52.17857142857143,S_4_52.17857142857143,C_5_52.17857142857143,S_6_52.17857142857143,C_7_52.17857142857143,S_8_52.17857142857143,C_9_52.17857142857143,S_10_52.17857142857143,C_11_52.17857142857143,S_12_52.17857142857143,C_13_52.17857142857143,S_14_52.17857142857143,C_15_52.17857142857143,S_16_52.17857142857143,C_17_52.17857142857143,S_18_52.17857142857143,C_19_52.17857142857143,S_20_52.17857142857143,C_21_52.17857142857143,S_22_52.17857142857143,C_23_52.17857142857143,S_24_52.17857142857143,C_25_52.17857142857143,S_26_52.17857142857143,C_27_52.17857142857143,S_28_52.17857142857143,C_29_52.17857142857143,S_30_52.17857142857143,C_31_52.17857142857143,S_32_52.17857142857143,C_33_52.17857142857143,S_34_52.17857142857143,C_35_52.17857142857143,S_36_52.17857142857143,C_37_52.17857142857143,S_38_52.17857142857143,C_39_52.17857142857143,S_40_52.17857142857143,C_41_52.17857142857143,S_42_52.17857142857143,C_43_52.17857142857143,S_44_52.17857142857143,C_45_52.17857142857143,S_46_52.17857142857143,C_47_52.17857142857143,S_48_52.17857142857143,C_49_52.17857142857143,S_50_52.17857142857143,C_51_52.17857142857143,S_0_4.34,C_1_4.34,S_2_4.34,C_3_4.34,id_cat,ds
0,-0.017202,0.999852,-0.034398,0.999408,-0.051584,0.998669,-0.068755,0.997634,-0.085906,0.996303,-0.103031,0.994678,-0.120126,0.992759,-0.137185,0.990545,-0.154204,0.988039,-0.171177,0.985240,-0.188099,0.982150,-0.204966,0.978769,-0.221772,0.975099,-0.238513,0.971139,-0.255182,0.966893,-0.271777,0.962360,-0.288291,0.957543,-0.304719,0.952442,-0.321058,0.947060,-0.337301,0.941397,-0.353445,0.935455,-0.369484,0.929237,-0.385413,0.922744,-0.401229,0.915978,-0.416926,0.908940,-0.432499,0.901634,-0.583090,-0.812407,0.947414,0.320011,1,2011-09-19
1,0.735034,0.678031,0.996751,-0.080549,0.616621,-0.787260,-0.160575,-0.987024,-0.834370,-0.551205,-0.970882,0.239557,-0.482206,0.876058,0.316982,0.948432,0.912053,0.410073,0.919817,-0.392347,0.335276,-0.942120,-0.465162,-0.885225,-0.966065,-0.258300,-0.844881,0.534955,-0.179645,0.983732,0.601271,0.799045,0.995005,0.099823,0.748017,-0.663680,0.019352,-0.999813,-0.721775,-0.692128,-0.998123,0.061245,-0.631740,0.775180,0.141444,0.989946,0.823547,0.567248,0.975336,-0.220724,0.499069,-0.866562,0.971604,0.236612,0.459786,-0.888030,1,2011-09-26
2,0.995218,-0.097683,-0.194431,-0.980916,-0.957232,0.289320,0.381441,0.924393,0.882712,-0.469915,-0.553893,-0.832588,-0.774500,0.632573,0.705204,0.709005,0.636728,-0.771089,-0.829598,-0.558361,-0.474653,0.880173,0.922329,0.386405,0.294462,-0.955663,-0.979857,-0.199702,-0.103031,0.994678,0.999986,0.005376,-0.092331,-0.995728,-0.981947,0.189155,0.284170,0.958774,0.926430,-0.376467,-0.465162,-0.885225,-0.835553,0.549409,0.628401,0.777890,0.712786,-0.701382,-0.767655,-0.640864,-0.562812,0.826585,-0.891335,0.453345,-0.808165,-0.588956,1,2011-10-03
3,0.589176,-0.808005,-0.952114,0.305743,0.949449,0.313921,-0.582205,-0.813042,-0.008601,0.999963,0.596104,-0.802907,-0.954709,0.297543,0.946714,0.322076,-0.575190,-0.818020,-0.017202,0.999852,0.602988,-0.797750,-0.957232,0.289320,0.943909,0.330207,-0.568133,-0.822937,-0.025801,0.999667,0.609827,-0.792534,-0.959686,0.281076,0.941034,0.338313,-0.561034,-0.827793,-0.034398,0.999408,0.616621,-0.787260,-0.962068,0.272811,0.938089,0.346394,-0.553893,-0.832588,-0.042993,0.999075,0.623370,-0.781927,0.381011,-0.924571,-0.704543,0.709661,1,2011-10-10
4,-0.211276,-0.977426,0.413013,0.910725,-0.596104,-0.802907,0.752283,0.658840,-0.874498,-0.485029,0.957232,0.289320,-0.996751,-0.080549,0.991269,-0.131859,-0.941034,0.338313,0.848314,-0.529494,-0.717295,0.696769,0.553893,-0.832588,-0.365484,0.930818,0.160575,-0.987024,0.051584,0.998669,-0.261414,-0.965227,0.459442,0.888208,-0.636728,-0.771089,0.785267,0.619157,-0.898354,-0.439273,0.970882,0.239557,-0.999579,-0.029025,0.983147,-0.182817,-0.922329,0.386405,0.819871,-0.572548,-0.680398,0.732843,0.313145,0.949705,0.594790,0.803881,1,2011-10-17


In [103]:
te =test.groupby(['StockCode'])['ds'].count().reset_index()

In [104]:
test.shape

(24197, 20)

In [105]:
test = test.loc[test['StockCode'].isin(te.loc[te['ds'] >= 7,'StockCode']),]

In [106]:
test['id_cat'] = id_encoder.transform(test['StockCode'])

In [107]:
test.loc[test['id_cat'].isin(sm.pr_sr_df['id_cat']),].shape

(24193, 20)

In [108]:
tr = train_val.groupby(['ds','StockCode'])[['ds','StockCode','Quantity']].tail(1).reset_index(drop=True)

In [109]:
test['ds'].max()

Timestamp('2011-10-31 00:00:00')

In [110]:
sm.pr_sr_df.groupby(['id_cat'])['ds'].max().unique()

array(['2011-11-07T00:00:00.000000000'], dtype='datetime64[ns]')

In [111]:
tr['id_cat'] = id_encoder.transform(tr['StockCode'])

In [112]:
# tr.merge(sm.sr_df,on=['id_cat','ds'],how='left')

In [175]:
(train_val.

Timestamp('2011-09-19 00:00:00')

# Recursive forecast implementation

In [176]:
def recursive_forecast(val,data,model,id_encoder,forecast_range=7,var='Quantity',exog_input=exog.columns.tolist(),forecast_for_validation=True):
  if forecast_for_validation:
    test_sr = sm.pr_sr_df.loc[sm.pr_sr_df['ds'] <= test['ds'].max(),]
  else:
    test_sr = sm.pr_sr_df.copy()
  val = val
  val['id_cat'] = id_encoder.transform(val['StockCode'])
  dat = val.loc[val['id_cat'].isin(test_sr['id_cat'].unique()),].copy()
  dat1 =data.copy()
  colname= dat.columns.tolist()
  for lag in range(2,5):
    dat['lag'+str(lag)] = dat.groupby(['StockCode'])[var].shift(lag)
  
  dat = dat.groupby(['StockCode'])[colname].tail(1).reset_index()
  da = dat.loc[dat['id_cat'].isin(test_sr['id_cat'].unique()),]
  da = da.merge(test_sr,on=['id_cat','ds'],how='left')
  notcol = [x for x in test_sr.columns if x not in ['id_cat','ds']]
  da = da[[x for x in da.columns if x not in [notcol]]]
  

    # da = 
  

  # test_sr = test_sr.
  # exog_input = 
  # print(da)
  da_f = pd.DataFrame()  
  for fr in range(1,forecast_range):
    
    da['ds'] = da['ds'] + pd.DateOffset(days=7)
    dst = da['ds'].max()
    print('date_max',dst)
    if fr < 5:
      for i in range(1,5):
        if i == 1:
          da['lag'+str(i)] = dat['forecast'].values
                   
        else:
          da['lag'+str(i+1)] = dat['lag'+str(i)].values
          
        if i == 4:
          datt = da['ds']-pd.to_timedelta(52, unit='w')
          # print(datt)
          dat12 = dat1.loc[dat1['ds'].isin(datt),['StockCode','ds']+[var]]
          # print(dat12)
          dat12 = dat12.rename(columns={var:'lag52'})
          da['ds52'] = datt.values
          da11 = da[['ds52','StockCode']].merge(dat12[['StockCode','ds','lag52']],left_on=['StockCode','ds52'],right_on=['StockCode','ds'],how='left')
          # print(da11)

          da['lag52'] = da11['lag52']
                  
          da['id_cat'] = id_encoder.transform(da['StockCode'])
          da = da.loc[da['id_cat'].isin(test_sr['id_cat'].unique()),]
          da1 = da[['Price', 'holiday', 'year', 'cumdist', 'averagedistance', 'cluster',
                    'wom', 'week_no', 'std', 'lag1', 'lag4', 'lag52']]
          
          da1.fillna(-1,inplace=True)

          model_in = rec_model_input(id_var='id_cat',exog=da1,val=False,val_ts=None,date=dst)
          
          da.loc[:,'forecast'] = model.predict(model_in)
          da['lforecast'] = np.where(da['forecast']*da['std']+da['mean'] < 0,0,da['forecast']*da['std']+da['mean'])
          da['cumsum'] = da['cumsum']+da['lforecast']
          da['cumdist'] = da.groupby(['StockCode','cumsum'])['timeelapsed'].cumsum()-1
          da.fillna(-1,inplace=True)
          print(da)
      da_f = pd.concat([da_f,da],axis=0)
    
    else:
      for j in range(1,5):
        if j == 1 :
           da['lag'+str(i)] = dat['forecast'].values
          
        elif j < 5:
          # print(da['lag'+str(j+1)])
          da['lag'+str(j+1)] = da['lag'+str(j)]
        if j == 4:
          datt = da['ds']-pd.to_timedelta(52, unit='w')
          # print(datt,'\n',da['ds'])
          dat12 = dat1.loc[dat1['ds'].isin(datt),['StockCode','ds']+[var]]
          dat12 = dat12.rename(columns={var:'lag52'})
          da['ds52'] =datt.values
          da11 = da[['ds52','StockCode']].merge(dat12[['StockCode','ds','lag52']],left_on=['StockCode','ds52'],right_on=['StockCode','ds'],how='left')
          # da11 = da11.rename(columns={})
          # print(da11)
          da['lag52'] = da11['lag52']
          
          # print(da)
          # print(da['lag365'])
          da['id_cat'] = id_encoder.transform(da['StockCode'])
          da = da.loc[da['id_cat'].isin(test_sr['id_cat'].unique()),]
          # id_ds = da['id_cat'].astype(str)+da['ds'].astype(str)
          # id_sr = 
          da1 = da[['Price', 'holiday', 'year', 'cumdist', 'averagedistance', 'cluster',
                    'wom', 'week_no', 'std', 'lag1', 'lag4', 'lag52']]
          sr = test_sr
          da1.fillna(-1,inplace=True)
          model_in = rec_model_input(id_var='id_cat',exog=da1,val=False,val_ts=None,date=dst)

          
          # print(da1)
          # print(da1.head())
          da.loc[:,'forecast'] = model.predict(model_in)
          da['lforecast'] = np.where(da['forecast']*da['std']+da['mean'] < 0,0,da['forecast']*da['std']+da['mean'])
          da['cumsum'] = da['cumsum']+da['lforecast']
          da['cumdist'] = da.groupby(['StockCode','cumsum'])['timeelapsed'].cumsum()-1
          da.fillna(-1,inplace=True)
          print(da)
          
      da_f = pd.concat([da_f,da],axis=0)
  return da_f
 

In [177]:
(pd.to_datetime('2011-09-12') + pd.DateOffset(days=7))-pd.to_timedelta(52, unit='w')

Timestamp('2010-09-20 00:00:00')

In [122]:
data1.min()

Quantity                             0
Price                        0.0514286
holiday                             -1
ds                 2010-01-04 00:00:00
year                              2010
timeelapsed                          0
cumsum                               0
cumdist                             -1
averagedistance                      0
cluster                              0
wom                                  0
week_no                              1
mean                                 0
std                                  0
tQuantity                     -1.91374
dtype: object

In [123]:
data.loc[data['ds']<'2010-09-20']

,StockCode,Quantity,Price,holiday,ds,year,timeelapsed,cumsum,cumdist,averagedistance,cluster,wom,week_no,mean,std,tQuantity,lag1,lag4,lag52
0,20677,96.000000,1.583599,-1.0,2010-05-31,2010,1.0,96.000000,0.0,0.013333,9,4,22,45.452000,40.057125,1.261898,-1.000000,-1.000000,-1.0
1,20677,10.500000,1.583599,-1.0,2010-06-07,2010,1.0,106.500000,0.0,0.013333,9,1,23,45.452000,40.057125,-0.872554,1.261898,-1.000000,-1.0
2,20677,32.000000,1.583599,-1.0,2010-06-14,2010,1.0,138.500000,0.0,0.013333,9,2,24,45.452000,40.057125,-0.335820,-0.872554,-1.000000,-1.0
3,20677,30.833333,1.583599,-1.0,2010-06-21,2010,1.0,169.333333,0.0,0.013333,9,3,25,45.452000,40.057125,-0.364945,-0.335820,-1.000000,-1.0
4,20677,123.666667,1.583599,-1.0,2010-06-28,2010,1.0,293.000000,0.0,0.013333,9,4,26,45.452000,40.057125,1.952578,-0.364945,1.261898,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305850,gift_0001_30,0.000000,25.420417,-1.0,2010-08-16,2010,1.0,22.000000,1.0,3.343750,14,2,33,0.578125,1.979272,-0.292090,0.213147,0.213147,-1.0
305851,gift_0001_30,0.000000,25.420417,-1.0,2010-08-23,2010,1.0,22.000000,2.0,3.343750,14,3,34,0.578125,1.979272,-0.292090,-0.292090,-0.292090,-1.0
305852,gift_0001_30,0.000000,25.420417,-1.0,2010-08-30,2010,1.0,22.000000,3.0,3.343750,14,4,35,0.578125,1.979272,-0.292090,-0.292090,-0.292090,-1.0
305853,gift_0001_30,10.000000,25.420417,-1.0,2010-09-06,2010,1.0,32.000000,0.0,3.343750,14,0,36,0.578125,1.979272,4.760274,-0.292090,0.213147,-1.0


In [124]:
train_f.head()

,Price,holiday,ds,year,cumdist,averagedistance,cluster,wom,week_no,std,tQuantity,lag1,lag4,lag52,id_cat
0,1.583599,-1.0,2010-05-31,2010,0.0,0.013333,9,4,22,40.057125,1.261898,-1.000000,-1.000000,-1.0,1
1,1.583599,-1.0,2010-06-07,2010,0.0,0.013333,9,1,23,40.057125,-0.872554,1.261898,-1.000000,-1.0,1
2,1.583599,-1.0,2010-06-14,2010,0.0,0.013333,9,2,24,40.057125,-0.335820,-0.872554,-1.000000,-1.0,1
3,1.583599,-1.0,2010-06-21,2010,0.0,0.013333,9,3,25,40.057125,-0.364945,-0.335820,-1.000000,-1.0,1
4,1.583599,-1.0,2010-06-28,2010,0.0,0.013333,9,4,26,40.057125,1.952578,-0.364945,1.261898,-1.0,1


In [125]:
fr = recursive_forecast(vale,data1,f_model,id_encoder,8,'tQuantity'forecast_for_validation=True)

date_max 2011-09-19 00:00:00
      level_0   index     StockCode  ...      lag5       ds52  lforecast
0      260986      67         20677  ... -0.086177 2010-09-20  37.860887
1      260992     163         20679  ...  1.614491 2010-09-20  21.288190
2      260998     259         20681  ... -0.742570 2010-09-20   8.954285
3      261004     355         20682  ...  2.731455 2010-09-20  16.534909
4      261010     451         20684  ... -0.585179 2010-09-20   4.844466
...       ...     ...           ...  ...       ...        ...        ...
3450   281692  305522             M  ... -0.337531 2010-09-20  16.735363
3451   281698  305618          PADS  ... -0.390184 2010-09-20   0.051639
3452   281704  305714          POST  ... -0.229442 2010-09-20  39.699782
3453   281710  305810  gift_0001_20  ... -0.319954 2010-09-20   0.065535
3454   281716  305906  gift_0001_30  ... -0.292090 2010-09-20   0.041444

[3455 rows x 84 columns]
date_max 2011-09-26 00:00:00
      level_0   index     StockCode  ...

In [126]:
fr.isnull().sum()

level_0      0
index        0
StockCode    0
Quantity     0
Price        0
            ..
S_2_4.34     0
C_3_4.34     0
lag5         0
ds52         0
lforecast    0
Length: 84, dtype: int64

In [127]:
# test = test.reset_index()

In [128]:
# train_val.loc[train_val['StockCode'].isin(t3.loc[t3['error'].isnull(),'StockCode']),]

In [129]:
test.isnull().sum()

StockCode          0
Quantity           0
Price              0
holiday            0
ds                 0
year               0
timeelapsed        0
cumsum             0
cumdist            0
averagedistance    0
cluster            0
wom                0
week_no            0
mean               0
std                0
tQuantity          0
lag1               0
lag4               0
lag52              0
id_cat             0
dtype: int64

In [130]:
test1 = test.merge(fr[['StockCode','ds','lforecast']],on=['StockCode','ds'],how='left')
# test1['forecast'] = test1['forecast']*test1['std'] + test1['mean']
test1['error'] = np.abs(test1['Quantity']-test1['lforecast'])/(test1['Quantity']+1)
t2 = test1.loc[test1['Quantity']>0,]
t = test1.groupby('StockCode')['error'].mean().reset_index()

In [131]:
test1.isnull().sum()

StockCode          0
Quantity           0
Price              0
holiday            0
ds                 0
year               0
timeelapsed        0
cumsum             0
cumdist            0
averagedistance    0
cluster            0
wom                0
week_no            0
mean               0
std                0
tQuantity          0
lag1               0
lag4               0
lag52              0
id_cat             0
lforecast          0
error              0
dtype: int64

In [132]:
t = t.sort_values(by=['error'])

In [133]:
t

,StockCode,error
1402,16047,0.000000
1436,16244A,0.000000
2365,35096A,0.000000
2781,84396,0.000000
2920,84753A,0.000000
...,...,...
1266,22444,53.772152
2470,40016,57.537813
2842,84568,91.860518
2763,84270,95.853000


In [134]:
t.groupby(['StockCode'])['error'].mean().mean()

2.3934306078432805

In [135]:
t.sort_values(by=['error'])

,StockCode,error
1402,16047,0.000000
1486,20621,0.000000
1370,10124A,0.000000
3184,85109,0.000000
2395,35838B,0.000000
...,...,...
1266,22444,53.772152
2470,40016,57.537813
2842,84568,91.860518
2763,84270,95.853000


In [136]:
t3  = t2.groupby('StockCode')['error'].mean().mean()

In [137]:
t3

1.0037302668381114

# Forecast mape saving for selection of best model

In [138]:
non_zero_mean_dpf = t2.groupby(['StockCode'])['error'].mean().reset_index()

In [139]:
non_zero_mean_dpf.to_csv('../../data/non-zero-mean_df.csv',index=False)

In [140]:
stock_cont = data1.groupby(['StockCode'])['Quantity'].sum()/data1['Quantity'].sum()

In [141]:
t = t.merge(stock_cont,on=['StockCode'],how='left')

In [142]:
t.sort_values(by=['Quantity'],ascending=False).head(60)

,StockCode,error,Quantity
389,84077,0.347324,0.008825
2989,17003,4.227893,0.007283
565,21212,0.430209,0.005959
1081,22492,0.676008,0.005075
3452,84568,91.860518,0.004837
2788,37410,3.203186,0.004460
1820,21977,1.235897,0.003986
596,22197,0.443977,0.003751
620,84991,0.455222,0.003651
2521,21980,2.363254,0.003643


In [111]:
test1.loc[test1['StockCode'].isin(t3.loc[t3['error'].isnull(),'StockCode']),'lforecast'] = test1.loc[test1['StockCode'].isin(t3.loc[t3['error'].isnull(),'StockCode']),'lforecast']

In [112]:
# len(t3[t3 <.2])

In [144]:
train_f2 = train_f[[x for x in train_f.columns if x in ['id_cat','ds','tQuantity']]]
val_f = val_f.loc[val_f['id_cat'].isin(train_f2['id_cat'].unique()),]
train_fexog = train_f[[x for x in train_f.columns if x not in ['id_cat','ds','tQuantity']]]
val_f2 = val_f[[x for x in val_f.columns if x in ['id_cat','ds','tQuantity']]]
val_fexog = val_f[[x for x in val_f.columns if x not in ['id_cat','ds','tQuantity']]]

# Forecast Creation

In [145]:
custom_object= {'swish':swish, 'mish':mish,'listh':listh,'sigmoid':'sigmoid','relu':'relu'}
sm =  SC.seasonal(train_f2,target=['tQuantity'],id_var='id_cat', horizon=8,k=None,daily=False,weekly=True, yearly=True,monthly= True,qtr=False,period=None,unit='w')
sm.seasonal_feature()
sm.seasonal_model(id_input=None,hidden =seasonal_hl,hidden_unit=seasonal_hunit, hidden_act=custom_object[sr_hidden_act],output_act=custom_object[sr_output_act],reg=seasonality_l1,
                  embed_size=50,drop_out=seasonal_dropout)

rm = RC.Regressor(ts=train_f2,exog=train_fexog,target=['tQuantity'],id_var='id_cat',categorical_var=categorical_feature,context_variable=context_variable, constraint=constraint,embed_size=50,
                  lat_unit=rm_lunit,lattice_size=rm_lsize,hidden_unit=rm_hunit,hidden_act=custom_object[rr_hidden_act], output_act=custom_object[rr_output_act],
                  hidden_layer=rm_hl,drop_out=rm_dropout,L1=rm_L1,rnge=.8)
train_y = train_f2['tQuantity'].values
val_y = val_f2['tQuantity'].values
rm.reg_model(id_input=sm.s_model.input[-1])
rc = rcm('additive')
rm2 = rc.Regressor_comp(sm.s_model.output,rm.combined_reg_model)
f_model = Model([sm.s_model.input,rm.combined_reg_model.input],rm2)
check_point_path = path='../../outputs/'
lr=0.0005
f_model.compile(loss=tf.keras.losses.MeanAbsolutePercentageError(),optimizer=tf.keras.optimizers.Adam(learning_rate=lr))
call_back_ = [ModelCheckpoint(check_point_path,monitor= 'val_loss',save_best_only=True, mode='min'),
            EarlyStopping(monitor='val_loss', mode='min', patience=10)
            ]
train_input = model_input([sm,rm],id_var='id_cat',exog=train_fexog)
val_input = model_input([sm,rm],id_var='id_cat',exog=val_fexog,val=True,val_ts=val_f2)

[26.  2.]
   id_cat         ds
0       1 2011-09-12
1       2 2011-09-12
2       3 2011-09-12
3       4 2011-09-12
4       5 2011-09-12
   id_cat         ds
0       1 2011-09-12
1       2 2011-09-12
2       3 2011-09-12
3       4 2011-09-12
4       5 2011-09-12
fts value        id_cat         ds
0           1 2011-09-19
1           1 2011-09-26
2           1 2011-10-03
3           1 2011-10-10
4           1 2011-10-17
...       ...        ...
27651    3457 2011-10-10
27652    3457 2011-10-17
27653    3457 2011-10-24
27654    3457 2011-10-31
27655    3457 2011-11-07

[27656 rows x 2 columns]
3457
KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.float32, name='id_cat'), name='id_cat', description="created by layer 'id_cat'")
KerasTensor(type_spec=TensorSpec(shape=(None, 50), dtype=tf.float32, name=None), name='reshape_15/Reshape:0', description="created by layer 'reshape_15'")
Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 

In [147]:
hist = f_model.fit(train_input,train_y,validation_data=(val_input,val_y),epochs=500,batch_size=2048,callbacks=call_back_)

Epoch 1/500
138/138 [==============================] - 7s 54ms/step - loss: 37139.9336 - val_loss: 542.1790
Epoch 2/500
138/138 [==============================] - 7s 52ms/step - loss: 27693.7070 - val_loss: 455.0382
Epoch 3/500
138/138 [==============================] - 7s 53ms/step - loss: 35695.3516 - val_loss: 509.1317
Epoch 4/500
138/138 [==============================] - 7s 53ms/step - loss: 22132.8398 - val_loss: 479.4186
Epoch 5/500
138/138 [==============================] - 7s 53ms/step - loss: 37062.7617 - val_loss: 432.1628
Epoch 6/500
138/138 [==============================] - 7s 54ms/step - loss: 14257.1758 - val_loss: 517.4761
Epoch 7/500
138/138 [==============================] - 7s 53ms/step - loss: 40853.0273 - val_loss: 451.5248
Epoch 8/500
138/138 [==============================] - 8s 61ms/step - loss: 37058.3789 - val_loss: 429.8630
Epoch 9/500
138/138 [==============================] - 8s 56ms/step - loss: 21597.8457 - val_loss: 401.2468
Epoch 10/500
138/138 [======

In [148]:
custom_object= {'swish':swish, 'mish':mish,'listh':listh}

In [149]:
f_model = tf.keras.models.load_model(path,custom_objects=custom_object) 

In [172]:
sm.pr_sr_df['ds'].max()

Timestamp('2011-12-26 00:00:00')

In [159]:
val_f1 = val.copy()
val_f1['id_cat'] = id_encoder.transform(val_f1['StockCode'])

In [160]:
val_f1 = val_f1.loc[val_f1['id_cat'].isin(val_f['id_cat'].unique()),]

In [161]:
val_f1.head()

,StockCode,Quantity,Price,holiday,ds,year,timeelapsed,cumsum,cumdist,averagedistance,cluster,wom,week_no,mean,std,tQuantity,lag1,lag4,lag52,id_cat
68,20677,116.166667,1.583599,-1.0,2011-09-19,2011,1.0,3163.066667,0.0,0.013333,9,2,38,45.452,40.057125,1.765346,-0.984893,-0.223481,-0.835107,1.0
69,20677,32.500000,1.583599,-1.0,2011-09-26,2011,1.0,3195.566667,0.0,0.013333,9,3,39,45.452,40.057125,-0.323338,1.765346,-0.435678,-0.627071,1.0
70,20677,156.000000,1.583599,-1.0,2011-10-03,2011,1.0,3351.566667,0.0,0.013333,9,0,40,45.452,40.057125,2.759759,-0.323338,-0.572982,-0.785179,1.0
71,20677,9.000000,1.583599,-1.0,2011-10-10,2011,1.0,3360.566667,0.0,0.013333,9,1,41,45.452,40.057125,-0.910000,2.759759,-0.984893,-0.772696,1.0
72,20677,18.833333,1.583599,-1.0,2011-10-17,2011,1.0,3379.400000,0.0,0.013333,9,2,42,45.452,40.057125,-0.664518,-0.910000,1.765346,-0.785179,1.0


In [162]:
val_f1['forecast'] = f_model.predict(val_input)

In [164]:
vale = val_f1.copy()
for lag in range(2,5):
  vale['lag'+str(lag)] = vale.groupby(['StockCode'])['tQuantity'].shift(lag)

In [167]:
vale.fillna(-1,inplace=True)

In [171]:
vale['ds'].max()

Timestamp('2011-10-31 00:00:00')

In [178]:
fr = recursive_forecast(vale,data,f_model,id_encoder,8,'tQuantity',forecast_for_validation=False)

date_max 2011-11-07 00:00:00
       index     StockCode    Quantity  ...      lag5       ds52  lforecast
0         74         20677   26.500000  ...  2.759759 2010-11-08  43.213311
1        170         20679   30.333333  ... -0.241501 2010-11-08  24.319526
2        266         20681    4.000000  ... -0.666632 2010-11-08   9.206053
3        362         20682    6.000000  ... -0.803858 2010-11-08  19.076440
4        458         20684    3.000000  ... -0.585179 2010-11-08   4.934719
...      ...           ...         ...  ...       ...        ...        ...
3450  305529             M    3.000000  ... -0.570719 2010-11-08  13.021715
3451  305625          PADS    0.000000  ... -0.390184 2010-11-08   0.056670
3452  305721          POST  372.333333  ... -0.277552 2010-11-08  35.885780
3453  305817  gift_0001_20    1.000000  ... -0.319954 2010-11-08   0.094927
3454  305913  gift_0001_30    0.000000  ...  0.213147 2010-11-08   0.049753

[3455 rows x 83 columns]
date_max 2011-11-14 00:00:00
    

In [179]:
forecast = fr[['ds','StockCode','lforecast']]

In [180]:
forecast['year'] = forecast['ds'].dt.year
forecast['Week No'] = forecast['ds'].dt.week

In [181]:
forecast = forecast.rename(columns={'lforecast':'Quantity'})

In [182]:
forecast[['StockCode','year','Week No','Quantity']].to_csv('../../outputs/deep_future_forecast.csv',index=False)